In [5]:
#二级结构特征、组成成分特征、位置特异性偏好特征、词向量编码
#PCA降维、ICA降维、KPCA降维、LLE降维
#这里的降维只是找出最好的降维策略，不是用来做最后的比较，不用在意性能变好了还是变差了

# 首先是二级结构特征

In [6]:
from features import ensembleFeature
import pandas as pd
import numpy as np
from utils import metricsCal
from thundersvm import SVC
from sklearn.model_selection import KFold
from features import ensembleFeature

In [54]:
def file_va(filename):
    f = open(filename,"rb")
    m=0
    name = []
    seq = []
    struc = []
    for i in f:
        #print(i)
        m+=1
        if m>2 and (m-2)%3==1:
            name.append(str(i[1:-1],'utf-8'))
        if m>3 and (m-2)%3==2:
            seq.append(str(i[:-1],'utf-8'))
        if m>3 and (m-2)%3==0:
            struc.append(str(i,'utf-8')[:41])
    return name,seq,struc

def RNAfold_dot(filename):
    f = open(filename,"rb")
    m = 0
    struc = []
    seq = []
    for i in f:
        m+=1
        if m%6==3:
            struc.append(str(i[0:41],'utf-8'))
        if m%6==2:
            seq.append(str(i[0:41],'utf-8'))
    return seq,struc

def struc_01(data):
    x = np.zeros(len(data)*2).reshape(-1,2)
    for i in range(len(data)):
        if data[i]=="(":
            x[i][0]=1
        elif data[i]==")":
            x[i][1]=1
    return x

def struc_01_8(data):
    x = np.zeros((len(data)-2)*8).reshape(-1,8)
    for i in range(len(data)-2):
        mm = 0
        if data[i]=="(" or data[i]==")":
            mm+=4
        if data[i+1]=="(" or data[i]==")":
            mm+=2
        if data[i+2]=="(" or data[i]==")":
            mm+=1
        x[i][mm]=1
    return x

##将序列转化为词向量用到的函数
def fa_seq(filepath):
    f = open(filepath,'r')
    x = []
    for i in f:
        x.append(i)
    seq = []
    for i in range(len(x)):
        if i%2==1:
            seq.append(x[i][:-1])
    return seq

In [ ]:
# train_name,train_seq,train_struc = file_va("data/mouse_train.log")
trainPosLabel = np.ones(int(len(train_seq)/2))
trainNegLabel = np.zeros(int(len(train_seq)/2))
trainLabel = np.append(trainPosLabel,trainNegLabel)
#获取01二级结构表示
trainData = []
for i in range(len(train_struc)):
    trainData.append(struc_01(train_struc[i]))
trainData = np.array(trainData)
        
#与单热编码结合
trainData_origin = np.append(trainData,ensembleFeature.Binary(train_seq),axis=2).reshape(trainData.shape[0],-1)

# 词向量嵌入特征，MultiLabel的

In [27]:
import torch
from torch import nn
import math
from utils import metricsCal
import pickle
import pandas as pd
from thundersvm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader,TensorDataset
import torch.nn.functional as F
import numpy as np
from Model import MultiLabel_fine_tuning as Model
import os
##将序列转化为词向量用到的函数
def fa_seq(filepath):
    f = open(filepath,'r')
    x = []
    for i in f:
        x.append(i)
    seq = []
    for i in range(len(x)):
        if i%2==1:
            seq.append(x[i][:-1])
    return seq

class EmbeddingSeq(nn.Module):
    def __init__(self,weight_dict_path):
        """
        Inputs:
            weight_dict_path: path of pre-trained embeddings of RNA/dictionary
        """
        super(EmbeddingSeq,self).__init__()
        weight_dict = pickle.load(open(weight_dict_path,'rb'))

        weights = torch.FloatTensor(list(weight_dict.values())).cuda()
        num_embeddings = len(list(weight_dict.keys()))
        embedding_dim = 300

        self.embedding = nn.Embedding(num_embeddings=num_embeddings,embedding_dim=embedding_dim)
        self.embedding.weight = nn.Parameter(weights)
        self.embedding.weight.requires_grad = False

    def forward(self,x):

        out = self.embedding(x.type(torch.cuda.LongTensor))

        return out

def seq2index(seqs,my_dict,window=3,save_data=False):
    """
    Convert single RNA sequences to k-mers representation.
        Inputs: ['ACAUG','CAACC',...] of equal length RNA seqs
        Example: 'ACAUG' ----> [ACA,CAU,AUG] ---->[21,34,31]
    """

    num_samples = len(seqs)
    temp = []
    for k in range(num_samples):
        length = len(seqs[k])
        seqs_kmers = [seqs[k][i:i+window] for i in range(0,length-window+1)]
        temp.append(seqs_kmers)


    seq_kmers = pd.DataFrame(data = np.concatenate(temp,axis=0))

    # load pretained word2vec embeddings

    word2index = word2index_(my_dict)

    seq_kmers_index = seq_kmers.applymap(lambda x: mapfun(x,my_dict))


    return seq_kmers_index.to_numpy()
def word2index_(my_dict):
    word2index = dict()
    for index, ele in enumerate(list(my_dict.keys())):
        word2index[ele] = index

    return word2index

def mapfun(x,my_dict):
    if x not in list(my_dict.keys()):
        return None
    else:
        return word2index_(my_dict)[x]

In [28]:
##得到train Label，trainData维度是1*39*300
embed = EmbeddingSeq('features/embeddings_12RM.pkl')
embeddings_dict = pickle.load(open('features/embeddings_12RM.pkl','rb'))
train_seq = fa_seq("data/Mouse/mouse_train.fasta")
trainLabel = np.append(np.ones(int(len(train_seq)/2)),np.zeros(int(len(train_seq)-len(train_seq)/2)),axis=0)
seq_T = []
for i in train_seq:
    seq_T.append(i.replace('U','T'))
x = seq2index(seq_T,embeddings_dict)
seqs_kmers_index = torch.transpose(torch.from_numpy(x),0,1)
trainData_origin = np.array(embed(seqs_kmers_index).view(len(train_seq),1,-1,300).cpu())

# 组成成分特征

In [35]:
trainData_EIIP = ensembleFeature.EIIP(train_seq)

trainData_PseEIIP = ensembleFeature.PseEIIP(train_seq)

trainData_PCP = ensembleFeature.PCP(train_seq)

trainData_NCPA = ensembleFeature.NCPA(train_seq)
trainData_NCPA = trainData_NCPA.reshape(trainData_NCPA.shape[0],-1)

trainData_DBPF = ensembleFeature.DBPF(train_seq)
trainData_DBPF = trainData_DBPF.reshape(trainData_DBPF.shape[0],-1)

trainData_origin = np.concatenate(  (trainData_EIIP,trainData_PseEIIP,trainData_PCP,trainData_NCPA,trainData_DBPF),axis=1)

# 位置特异性偏好特征

In [95]:
train_seq = fa_seq("data/Mouse/mouse_train.fasta")
test_seq = fa_seq("data/Mouse/mouse_indep.fasta")
trainPos_seq = train_seq[0:int(len(train_seq)/2)]
trainNeg_seq = train_seq[int(len(train_seq)/2):]
testPos_seq = test_seq[0:int(len(test_seq)/2)]
testNeg_seq = test_seq[int(len(test_seq)/2):]
trainLabel = np.append(np.ones(int(len(train_seq)/2)),np.zeros(int(len(train_seq)-len(train_seq)/2)),axis=0)
testLabel = np.append(np.ones(int(len(test_seq)/2)),np.zeros(int(len(test_seq)-len(test_seq)/2)),axis=0)

#首先把所有特征表示出来
#值
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSNP = np.vstack((trainPos,trainNeg))
testData_PSNP = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSDP = np.vstack((trainPos,trainNeg))
testData_PSDP = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,3) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSTP = np.vstack((trainPos,trainNeg))
testData_PSTP = np.vstack((testPos,testNeg))
trainData_PSKP = np.hstack((trainData_PSNP,trainData_PSDP,trainData_PSTP))
testData_PSKP = np.hstack((testData_PSNP,testData_PSDP,testData_PSTP))



trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,0)
trainData_PSCP_0 = np.vstack((trainPos,trainNeg))
testData_PSCP_0 = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1)
trainData_PSCP_1 = np.vstack((trainPos,trainNeg))
testData_PSCP_1 = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2)
trainData_PSCP_2 = np.vstack((trainPos,trainNeg))
testData_PSCP_2 = np.vstack((testPos,testNeg))

trainData_PSCP = np.hstack((trainData_PSCP_0,trainData_PSCP_1,trainData_PSCP_2))
testData_PSCP = np.hstack((testData_PSCP_0,testData_PSCP_1,testData_PSCP_2))

trainPos,trainNeg,testPos,testNeg = ensembleFeature.BPB(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq)
trainData_BPB = np.vstack((trainPos,trainNeg))
testData_BPB = np.vstack((testPos,testNeg))

trainData_KNN = np.array(pd.read_csv("features/KNN_value_train.csv").iloc[:,1:])
testData_KNN = np.array(pd.read_csv("features/KNN_value_test.csv").iloc[:,1:])

trainData_origin = np.hstack((trainData_PSKP,trainData_PSCP,trainData_BPB))#,trainData_KNN))
testData = np.hstack((testData_PSKP,testData_PSCP,testData_BPB))#,testData_KNN))

# 下面是降维四件套

In [34]:
trainData = trainData_origin.reshape(9126,-1)

In [37]:
trainData.shape

(9126, 800)

In [36]:
#主成分分析降维  sec_PCA  com_PCA loc_PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=800)  # 加载PCA算法，设置降维后主成分数目为2
pca.fit(trainData)  # 对样本进行降维
trainData = pca.transform(trainData)
#testdata_x = pca.transform(testData)

In [96]:
#ICA降维  sec_ICA  com_ICA loc_ICA
from sklearn.decomposition import FastICA
ica = FastICA(n_components= 100).fit(trainData_origin)
trainData = ica.transform(trainData_origin)

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


In [66]:
#KPCA降维  sec_KPCA  com_KPCA loc_KPCA
from sklearn.decomposition import KernelPCA
scikit_kpca = KernelPCA(n_components=100, kernel='linear', gamma=3)
scikit_kpca.fit(trainData_origin)
trainData = scikit_kpca.transform(trainData_origin)

In [69]:
#LLE降维  sec_LLE  com_LLE loc_LLE
from sklearn.manifold import LocallyLinearEmbedding
lle = LocallyLinearEmbedding(n_components=100, n_neighbors=50, random_state=42)
lle.fit(trainData_origin)
trainData = lle.transform(trainData_origin)

In [38]:
#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
#     ica = FastICA(n_components= 100)
#     ica.fit(X_train)
#     X_train = ica.transform(X_train)
#     X_test = ica.transform(X_test)
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
#     _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(testdata_x), testLabel,th )
    print(auc)
wv_PCA = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(wv_PCA[4])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7977145881447267
0.8014374741936412
0.8176311343214807
0.7987520998320135
0.8181006423366074
0.8090493018944442
0.7870923598797317
0.8218048909514992
0.798684185214397
0.8113396358038054


0.8061606312572348

In [80]:
sec_PCA  com_PCA loc_PCA
sec_ICA  com_ICA loc_ICA
sec_KPCA  com_KPCA loc_KPCA
sec_LLE  com_LLE loc_LLE

array([[0.7875    , 0.7690583 , 0.8125    , 0.81777778, 0.77896996,
        0.79097387, 0.77521008, 0.79120879, 0.74454148, 0.81431767],
       [0.69515012, 0.70877944, 0.67483296, 0.66954644, 0.74049217,
        0.68902439, 0.67889908, 0.69584245, 0.70704846, 0.63225806],
       [0.74370208, 0.73822563, 0.74479737, 0.74260679, 0.76013143,
        0.73603505, 0.72916667, 0.74342105, 0.72587719, 0.72149123],
       [0.48535485, 0.47829264, 0.49252105, 0.49221274, 0.51998161,
        0.47954701, 0.4567679 , 0.48923008, 0.45193783, 0.45334392],
       [0.81109988, 0.81508244, 0.82542143, 0.81600672, 0.83039529,
        0.82267346, 0.79500424, 0.82671027, 0.80282977, 0.81378365]])

In [85]:
a = np.vstack( (np.mean(sec_PCA,axis=1),np.mean(sec_ICA,axis=1),np.mean(sec_KPCA,axis=1),np.mean(sec_LLE,axis=1),
np.mean(com_PCA,axis=1),np.mean(com_ICA,axis=1),np.mean(com_KPCA,axis=1),np.mean(com_LLE,axis=1),
np.mean(loc_PCA,axis=1),np.mean(loc_ICA,axis=1),np.mean(loc_KPCA,axis=1),np.mean(loc_LLE,axis=1)))

lista = ["sec_PCA","sec_ICA","sec_KPCA","sec_LLE","com_PCA","com_ICA","com_KPCA","com_LLE","loc_PCA","loc_ICA","loc_KPCA","loc_LLE"]

In [89]:
#pd.DataFrame(np.append(np.array(lista).reshape(-1,1),a,axis=1)).to_csv("Result/sec_com_loc_reduce_dimension.csv",header=["name","Sen","Spe","Acc","Mcc","Auc"])

In [110]:
Auc

array([0.70347623, 0.70605237, 0.7111661 , 0.70309575, 0.69704804,
       0.72033051, 0.68024584, 0.68387477, 0.70729373, 0.69531645])